In [ ]:
print('Comenzando Instalacion')
from IPython.utils import capture
with capture.capture_output() as cap:
  from google.colab import drive
  drive.mount('/content/drive')
  #
  !mkdir /content/log
  !mkdir /content/drive/MyDrive/train
  !mkdir /content/drive/MyDrive/train/img
  !mkdir /content/drive/MyDrive/train/output
  !mkdir -p /root/.cache/huggingface/accelerate/
  #
  %cd /content
  !curl -Lo /content/zeroLR.zip https://huggingface.co/josemerinom/app/resolve/main/zeroLR_0_7_0.zip
  !unzip -u /content/zeroLR.zip
  #
  %cd /content/zeroLR
  #
  !pip install -r req1.txt
  !pip install -r req2.txt
  !pip install -r req3.txt
  !pip install -e .
  !pip install -r req4.txt
  #
  %cd /content
  !curl -Lo /content/default_config.yaml https://huggingface.co/josemerinom/app/resolve/main/default_config_24_fp16.yaml
  !cp -r /content/default_config.yaml /root/.cache/huggingface/accelerate/
print('Instalacion Finalizada')

In [ ]:
def train(name,dataset):
  l_name=name
  l_dataset=f"/content/drive/MyDrive/train/{dataset}"
  %cd /content/zeroLR
  !accelerate launch train_network.py \
    --caption_extension=".txt" \
    --clip_skip=1 \
    --gradient_accumulation_steps=1 \
    --keep_tokens=1 \
    --learning_rate=0.0001 \
    --logging_dir="/content/log" \
    --lr_scheduler_type="CosineAnnealingLR" \
    --lr_scheduler_args "T_max=150" \
    --max_data_loader_n_workers=0 \
    --max_token_length=225 \
    --max_train_epochs=5 \
    --mixed_precision="fp16" \
    --network_alpha=80 \
    --network_dim=96 \
    --network_module=networks.lora \
    --no_half_vae \
    --optimizer_type="AdamW" \
    --optimizer_args "betas=(0.9, 0.999)" "eps=1e-8" "weight_decay=0.01" \
    --output_dir="/content/drive/MyDrive/train/output" \
    --output_name=$l_name \
    --pretrained_model_name_or_path="josemerinom/zero15" \
    --prior_loss_weight=1 \
    --resolution="512,512" \
    --save_every_n_epochs=1 \
    --save_model_as=safetensors \
    --save_precision="fp16" \
    --scale_weight_norms=1 \
    --seed=1 \
    --text_encoder_lr=0.00008 \
    --train_batch_size=1 \
    --train_data_dir=$l_dataset \
    --unet_lr=0.0001 \
    --vae_batch_size=1 \
  #
train("lora_v01","img")

In [ ]:
#desconectar
print('Done')
from google.colab import runtime
import time
time.sleep(60)
runtime.unassign()